#Install and Import Packages

In [0]:
!pip install sumy
!pip install rouge-score

In [0]:
import pandas as pd
import sumy
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

#Load and Preprocess Data

In [0]:
all_the_news_df = pd.read_csv("/content/drive/My Drive/CIS 522/Project/all_the_news_comma.csv")

In [0]:
all_the_news_df = all_the_news_df.dropna(how='any', axis=0)

In [0]:
all_the_news_df['content_length'] = all_the_news_df['content'].apply(lambda x: len(x))
all_the_news_df['title_length'] = all_the_news_df['title'].apply(lambda x: len(x))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [0]:
all_the_news_df = all_the_news_df[(all_the_news_df['content_length'] > 10) & (all_the_news_df['title_length'] > 5)]
all_the_news_df['content'] = all_the_news_df['content'].apply(lambda x: x.replace('[', '').replace(']', ''))

In [0]:
from sklearn.model_selection import train_test_split
content = all_the_news_df['content']
title = all_the_news_df['title']
content_train, content_test, title_train, title_test = train_test_split(content, title, test_size=0.2, random_state=42)

In [0]:
content_test.reset_index(inplace=True, drop=True)

In [0]:
title_test.reset_index(inplace=True, drop=True)

In [0]:
# import pickle
# pickle_in = open("/content/drive/My Drive/CIS 522/Project/all_the_news_df_clean.pickle","rb")
# all_the_news_df_raw = pickle.load(pickle_in)

In [0]:
# all_the_news_df = all_the_news_df_raw[["title", "content"]]

In [0]:
# train = pd.read_csv("/content/drive/My Drive/CIS 522/Project/news_subset_train.csv")
# test = pd.read_csv("/content/drive/My Drive/CIS 522/Project/news_subset_test.csv")

In [0]:
# test['content'] = test['content'].astype('str')
# test['title'] = test['title'].astype('str')

In [0]:
# test['content_length'] = test['content'].apply(lambda x: len(x))
# test['title_length'] = test['title'].apply(lambda x: len(x))

In [0]:
# test = test[(test['content_length'] > 10) & (test['title_length'] > 5)]


In [0]:
# test['content'] = test['content'].apply(lambda x: x.replace('[', '').replace(']', ''))

In [0]:
# test.reset_index(drop=True, inplace=True)

In [0]:
!mkdir /content/txt_files

In [0]:
for i in range(len(content_test)):
  file_name = 'content_' + str(i) + '.txt'
  with open('/content/txt_files/' + file_name, 'w') as file:
    file.write(content_test[i])
    file.close()

#Get Summarization and Compute BLEU Score

In [0]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer as Summarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

LANGUAGE = 'english'
SENTENCES_COUNT = 1
stemmer = Stemmer(LANGUAGE)
summarizer = Summarizer(stemmer)
summarizer.stop_words = get_stop_words(LANGUAGE)
file_path = '/content/txt_files/'

BLEUScore_list = []

for i in range(len(content_test)):
  filename = file_path + 'content_' + str(i) + '.txt'
  parser = PlaintextParser.from_file(filename, Tokenizer(LANGUAGE))

  for sentence in summarizer(parser.document, SENTENCES_COUNT):
    summary_hypothesis = str(sentence)
  if isinstance(summary_hypothesis, list):
    summary_hypothesis = 'This summary is wrong'

  
  summary_reference = title_test[i]
  print(i)
  print(summary_hypothesis)
  print(summary_reference)
  print('--------------------')
  summary_hypothesis = summary_hypothesis.split()
  summary_reference = summary_reference.split()

  BLEUScore = sentence_bleu([summary_reference], summary_hypothesis, smoothing_function=SmoothingFunction().method1)
  BLEUScore_list.append(BLEUScore)

Streaming output truncated to the last 5000 lines.
"Individuals can get on a plane and cross great distances in a short period of time," says Dr. Nikos Vasilakis, a professor in the department of pathology at the University of Texas Medical Branch in Galveston.
Mapping Zika: From A Monkey In Uganda To A Growing Global Concern
--------------------
18359
Such a wheeler-dealer is no more bothered by an anchorman’s raised brow than he was by a banker’s frown., How does the establishment derail an out-of-control train for whom there are no gaffes, who has no fear of the New York Times, who offers no apologies for speaking what much of the country thinks — and who apparently needs neither money from Republicans nor politically correct approval from Democrats?, — Victor Davis Hanson is a classicist and historian at the Hoover Institution, Stanford University, and the author, most recently, of The Savior Generals.
Obama’s Failings Are the Reasons for Trump’s Rise
--------------------
18360
Emb

/usr/local/lib/python3.6/dist-packages/sumy/summarizers/lsa.py:76: UserWarning: Number of words (1073) is lower than number of sentences (1138). LSA algorithm may not work properly.
  warn(message % (words_count, sentences_count))


Streaming output truncated to the last 5000 lines.
--------------------
26753
When Heidi and Ted Cruz came to the Atlanta suburbs to campaign a few weeks ago, she perfectly played the role of political spouse: a loving gaze for her husband as he spoke, hands on the shoulders of their two daughters, beaming smiles for supporters.
Heidi Cruz is the high-powered political spouse we've rarely seen since Hillary Clinton
--------------------
26754
MOST POPULARThis week on “The Tom Joyner Morning Show,” MSNBC host Al Sharpton said Vice President Mike Pence was a “true believer in alt-right stuff” and a “true enemy” of civil rights.
Sharpton: Pence Is a 'True Enemy' of Civil Rights - Breitbart
--------------------
26755
(Additional reporting by Ahmed Rasheed and Saif Hameed in Baghdad; Editing by Louise Ireland) QAMISHLI, Syria/ANKARA The head of the Syrian Kurdish YPG militia said on Wednesday that Turkish military deployments near Kurdish-held areas of northwestern Syria amounted to a "decla

/usr/local/lib/python3.6/dist-packages/sumy/summarizers/lsa.py:76: UserWarning: Number of words (677) is lower than number of sentences (701). LSA algorithm may not work properly.
  warn(message % (words_count, sentences_count))


Streaming output truncated to the last 5000 lines.
It’s interesting that Trump alluded to the need to do something about the recent police shooting deaths in Louisiana and Minnesota — meaning the killings of Alton Sterling and Philando Castile, both black men — particularly in the context of his suggestion that we must restore law and order via strong leadership.
After assassination of police officers, Trump says the right thing
--------------------
35612
", "There is no place for this heinous and divisive behavior in our city," he said in a statement., McClinton, who is from the Greenville area,  is a parishioner of the church of about 200 congregants, said Simmons' chief of staff, Kenya Collins.
Parishioner charged in Mississippi church fire
--------------------
35613
To continue to deny that – it just flies in the face of all of the reality.”, Flynn was forced to resign earlier this year after it was revealed he lied to Mike Pence about the nature of his conversations with Sergey Ki

In [0]:
import numpy
from statistics import mean, median
mean(BLEUScore_list)

0.005460184928179186

In [0]:
median(BLEUScore_list)

0.004185635735742376